In [ ]:
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import ipywidgets as widgets
from google.colab import files

from random import randint
import random
import os
import io
from google.colab import drive

from IPython.display import HTML, display
from matplotlib import animation as F
from matplotlib import pyplot as plt
import matplotlib

%matplotlib inline

url = 'https://raw.githubusercontent.com/Abeni18/COVID-visualization-/master/COVID_dataset/covid_19_data.csv'
covid = pd.read_csv(url, error_bad_lines=False)
covid.rename(columns={'ObservationDate':'Date','Province/State':'State','Country/Region':'Country'},inplace=True)
covid=covid[["Date","State","Country","Confirmed","Deaths","Recovered"]]
class Covid_dynamic_graph():
  def __init__(self, data,country='US', viewBy="Country", case='Confirmed',speed=250, animation_ = 'dsplay', file_path='animation_by_country.mp4'):
    self.fig, self.ax = plt.subplots(figsize=(20,12)) #Set figure for plot
    plt.close()

    self.case = case
    self.viewBy = viewBy
    self.speed = speed
    self.animation_= animation_
    self.file_path = file_path
    if self.viewBy == 'State':
      self.country = country
      self.data = data[data.Country == country]
    elif self.viewBy == 'Country':
      self.country = 'World'
      self.data = data
      
  def draw_barchart_state(self, date):
      data = self.data
      case = self.case
      if self.viewBy == "State":
        catagory = "State"
      elif self.viewBy == "Country":
        catagory = "Country"

      grouped = data.groupby([catagory,'Date'])
      covid_confirmed = grouped.sum().reset_index().sort_values(['Date'],ascending=False)
      dff_total = (covid_confirmed[covid_confirmed['Date'].eq(date)].sort_values(by=case,ascending=False))
      dff = dff_total.head(10)[::-1]
      # calculating the cases value
      Total_confirmed = sum(dff_total['Confirmed'])
      Total_death = sum(dff_total['Deaths'])
      Total_recovered = sum(dff_total['Recovered'])
      Total_recovered_prec = f'{(Total_recovered/Total_confirmed)*100:,.0f}'
      Total_death_prec =  f'{(Total_death/Total_confirmed)*100:,.0f}'
      self.ax.clear()

      # creating the dynamic bar graph
      self.ax.barh(dff[catagory], dff[case], color=["#980501","#CD1212","#D84E4E","#CB6262","#D39B5F","#F7EC10","#D0F710","#9CF710","#B4D67F","#969C8E"][::-1])
      dx = dff[case].max() / 200
      for i, (value, name) in enumerate(zip(dff[case], dff[catagory])):
          self.ax.text(value-dx, i,     name,           size=16, weight=600, ha='right', va='center')
          self.ax.text(value+dx, i,     f'{value:,.0f}',  size=16, weight=500, ha='left',  va='center')

          self.ax.text(1, 0.4, 'Date: ' + date, transform=self.ax.transAxes, color='#777777', size=18, ha='right', weight=600)
          self.ax.text(1, 0.3, 'Total confirmed: '+ str(f'{Total_confirmed:,.0f}'), transform=self.ax.transAxes, color='#777777', size=18, ha='right', weight=600)
          self.ax.text(1, 0.2, 'Total recovered: ' + str(f'{Total_recovered:,.0f}')  + '  '+ str(Total_recovered_prec) + ' %', transform=self.ax.transAxes, color='#777777', size=18, ha='right', weight=600)
          self.ax.text(1, 0.1, 'Total death: ' +  str(f'{Total_death:,.0f}') + '  '+ str(Total_death_prec) + ' %', transform=self.ax.transAxes, color='#777777', size=18, ha='right', weight=600)
    
     
      self.ax.text(0, 1.06, 'Number of ' + self.case + " cases", transform=self.ax.transAxes, size=12, color='#777777')
      self.ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
      self.ax.xaxis.set_ticks_position('top')
      self.ax.tick_params(axis='x', colors='#777777', labelsize=16)
      self.ax.set_yticks([])
      self.ax.margins(0, 0.01)
      self.ax.grid(which='major', axis='x', linestyle='-')
      self.ax.set_axisbelow(True)
      self.ax.text(0, 1.12, 'The number of '+str(case)+' Cases in the'+ self.country + ' upto ' + str(date) + ' by ' + str(catagory),
              transform=self.ax.transAxes, size=24, weight=600, ha='left')
      self.ax.text(1, 0, 'Created by @ Abenezer Girma (aggirma@aggies.ncat.edu)', transform=self.ax.transAxes,size=18, ha='right',
              color='#777777', bbox=dict(facecolor='white', alpha=0.96, edgecolor='black'))
      plt.box(False)

  def data_preporocess(self):
      # Create the animation 
      animator= F.FuncAnimation(self.fig,self.draw_barchart_state,frames=self.data.Date.unique(),interval=self.speed) #Building animation
      matplotlib.rcParams['animation.embed_limit'] = 2**128
      if self.animation_ == 'display':
        return HTML(animator.to_jshtml())
      elif self.animation_ == 'save':
        drive.mount('/content/gdrive')
        path = F"/content/gdrive/My Drive/"+self.file_path 
        Writer = F.writers['ffmpeg']
        writer = Writer(fps=6, metadata=dict(artist='Abenezer'), bitrate=2800)
        return animator.save(path, writer=writer)
us_covid = Covid_dynamic_graph(data=covid, animation_ = 'display')
us_covid.data_preporocess()